In [1]:
from main import process_input, retrieve_3d
import open3d as o3d
from glob import glob
import os.path as osp
from utils.misc import load_config
import open_clip
import os
import pickle
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def correct_off_header(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    if len(lines[0].split('OFF')) > 1:
        header = lines[0].split('OFF')
        lines[0] = 'OFF\n'
        lines.insert(1, ' '.join(header[1:]).strip() + '\n')

    with open(file_path, 'w') as file:
        file.writelines(lines)

In [3]:
config = load_config()

print("loading OpenCLIP model...")
os.makedirs('./clip_cache', exist_ok=True)

open_clip_model, _, open_clip_preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k', cache_dir='./clip_cache')
open_clip_model = open_clip_model.cpu().eval()
device = 'cpu'

print('loading Shape Embeddings...')

with open('./modelnet_embed/modelnet.pkl', 'rb') as f:
    shape_embeddings = pickle.load(f)
    shape_ids = np.array(list(shape_embeddings.keys())) # (N,)
    embeddings = np.array(list(shape_embeddings.values())) # (N, embed_dim)


while True:
    user_input = input("Enter a user description of shape to retrieve: ")
    k = int(input("Enter the number of shapes to retrieve: "))
    refined_text = [user_input]

    text_feature = process_input(refined_text, open_clip_model, device)[0] # (1, embed_dim,)
    retrieve_3d(text_feature, embeddings, shape_ids, config, k=k)

    results = glob("results/*.off")

    for result in results:
        file_name = osp.basename(result).split(".")[0]
        correct_off_header(result)
        mesh = o3d.io.read_triangle_mesh(result)
        o3d.visualization.draw_geometries([mesh], window_name=file_name)

loading OpenCLIP model...
loading Shape Embeddings...


ValueError: invalid literal for int() with base 10: ''